In [47]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
from typing import List, NamedTuple, Union, Literal, Union, Dict, Any
import random
import re
from pathlib import Path
from collections import deque
import fitz # PyMuPDF is imported as fitz
from pydantic import BaseModel, Field

from lib.translate import translate

from lib.pdf.models import Page
from lib.pdf.operations import reduce_page
from lib.pdf.utils import view_doc, do_bounding_boxes, do_translation_redaction
from lib.pdf.font import get_font

In [49]:
# Subsets:
# subset("./docs/ulykker.pdf", "./docs/ulykker-subset.pdf", [18,19])

In [50]:
document_path = "./docs/ulykker-subset.pdf"

# Bounding Boxes

In [51]:
# Load Document
doc = fitz.open(document_path) 

pages = doc.pages()
pages = map(lambda page: page.get_text("dict"), pages)
pages = map(Page.model_validate, pages)
pages = map(reduce_page, pages)
pages = list(pages)
view_doc(pages)

PAGE
	Block
 Børneulykker i Danmark 
		Line
			Span: 'Børneulykker i Danmark '
	Block
 Børns ulykker i tal, trends og temaer
		Line
			Span: 'Børns ulykker i tal, trends og temaer'
	Block
 Der var i perioden 2000-2004 i gennemsnit årligt ca. 170.000 skadestuebesøg blandt børn i Danmark i alderen 0-14 år, beregnet ud fra Ulykkesregisteret (2000-2004). Heraf blev ca. 6000 indlagt. Ca. 50 børn døde årligt som følge af ulykker (Dødsårsagsregisteret, 1995-2000). Langt de fleste skader blandt børn er altså mindre alvorlige, og kun ganske få medfører døden. Dette er illustreret med ulykkespyramiden (fig. 1). Børn bidrager med 27% af alle skadestuebesøg i forbindelse med ulykker, og 14% af indlæggelserne. Børn udgør kun ca. 2% af dødsulykkerne, men ulykker er alligevel den hyppigste dødsårsag for børn over 1 år og er skyld i omkring hvert tredje dødsfald i aldersgruppen 1-14 år. 
		Line
			Span: 'Der var i perioden 2000-2004 i gennemsnit årligt '
		Line
			Span: 'ca. 170.000 skadestuebesøg bla

In [52]:
do_bounding_boxes(doc, pages)
# Save modified document
doc_path = Path(document_path)
annotated_path =str(doc_path.parent / "bbox" / doc_path.name)
doc.save(annotated_path)

# Translation

In [53]:
# Load Document
doc = fitz.open(document_path) 

pages = doc.pages()
pages = map(lambda page: page.get_text("dict"), pages)
pages = map(Page.model_validate, pages)
pages = map(reduce_page, pages)
pages = list(pages)

In [54]:
for page in pages:
    for block in page.blocks:
        print("Original text:", block.text)
        block.text = translate(block.text)
        print("Translated text:", block.text)

Original text:  Børneulykker i Danmark 
Translated text: Child accidents in Denmark
Original text:  Børns ulykker i tal, trends og temaer
Translated text: Children's accidents in numbers, trends and themes
Original text:  Der var i perioden 2000-2004 i gennemsnit årligt ca. 170.000 skadestuebesøg blandt børn i Danmark i alderen 0-14 år, beregnet ud fra Ulykkesregisteret (2000-2004). Heraf blev ca. 6000 indlagt. Ca. 50 børn døde årligt som følge af ulykker (Dødsårsagsregisteret, 1995-2000). Langt de fleste skader blandt børn er altså mindre alvorlige, og kun ganske få medfører døden. Dette er illustreret med ulykkespyramiden (fig. 1). Børn bidrager med 27% af alle skadestuebesøg i forbindelse med ulykker, og 14% af indlæggelserne. Børn udgør kun ca. 2% af dødsulykkerne, men ulykker er alligevel den hyppigste dødsårsag for børn over 1 år og er skyld i omkring hvert tredje dødsfald i aldersgruppen 1-14 år. 
Translated text: In the period 2000-2004, average annually was about 170,000 injur

In [55]:
do_translation_redaction(doc, pages)
# Save modified document
doc_path = Path(document_path)
annotated_path =str(doc_path.parent / "translated" / doc_path.name)
doc.save(annotated_path)